**Input Datasets**
1. 'landfilllmopdata.xlsx' This dataset is taken from: https://www.epa.gov/lmop/project-and-landfill-data-state and describes all United States Landfills

**Output Datasets**
1. "Landfill.csv" This dataset is comprised of all "open" United States landfills. Added data includes additional data that is calculated from the LMOP data: 'Designed Volume (acres^3)', 'Current Volume Current (acres^3)', 'Percent Volume Available', 'Years until Fill', 'Years to Planned Closure (from 2023)', and 'Difference in Years to fill and planned closure'.
2. "No Years to Fill.csv" This is a csv of all landfills which did not have enough data to estimate at which year they will reach capacity at the current fill rate.
3. 'Landfills to Close or Full in <= 10 Years.csv' Dataset which reports all landfills which are either planned to close in less than 10 years or will reach capacity.
4. 'State Closure Counts (<=10 years).csv' This csv counts the number of landfills which will close in the next ten years for each state.

In [ ]:
import pandas as pd

In [ ]:

df = pd.read_excel('landfilllmopdata.xlsx', sheet_name='LMOP Database', header=0)
df = df[['County',
         'State',
         'Longitude',
         'Latitude',
         'Landfill Closure Year',
         'Current Landfill Status',
         'Design Landfill Area (acres)',
 'Current Landfill Area (acres)',
 'Design Landfill Depth (feet)',
 'Current Landfill Depth (feet)',
 'Landfill Design Capacity (tons)',
 'Waste in Place (tons)',
         'Annual Waste Acceptance Rate (tons per year)',]]

#add columns for percent volume available convert feet to acres using 1foot is 2.29568e-5 acres
df['Designed Volume (acres^3)'] = df['Design Landfill Area (acres)'] * df['Design Landfill Depth (feet)']*2.29568e-5
df['Current Volume Current (acres^3)'] = df['Current Landfill Area (acres)'] * df['Current Landfill Depth (feet)']*2.29568e-5
df['Percent Volume Available'] = 1 - df['Current Volume Current (acres^3)']/df['Designed Volume (acres^3)']
# df

In [ ]:
##extracting only the open landfills

df_open = df[df['Current Landfill Status'] == 'Open'].copy()

#additional calculations for
df_open['Years until Fill'] = round((df_open['Landfill Design Capacity (tons)'] - df_open['Waste in Place (tons)'])  / df_open['Annual Waste Acceptance Rate (tons per year)'],2)
df_open['Years to Planned Closure (from 2023)'] = round(df_open['Landfill Closure Year'] - 2023)
df_open['Difference in Years to fill and planned closure'] = df_open['Years until Fill'] - df_open['Years to Planned Closure (from 2023)']
# df_fillbeforeclosed = df_open[df_open['Difference in Years to fill and planned closure'] < 0].copy()
# df_fillbeforeclosed


df_open.to_csv("Landfill.csv")

In [ ]:

#extracting all landfills which will close in 10 years
below_10 = df_open[(df_open['Years to Planned Closure (from 2023)'] <= 10) | (df_open['Years until Fill'] <= 10)].copy()
below_10.to_csv('Landfills to Close or Full in <= 10 Years.csv')

#count the number of landfills closing in 10 years per state
counts = below_10['State'].value_counts().reset_index()
counts.columns = ['State', 'Number of Landfills Closing in the Next 10 Years']
counts.groupby('Number of Landfills Closing in the Next 10 Years')
counts.to_csv('State Closure Counts (<=10 years).csv')

#which states do not have landfills closing in the next ten years?
not_in_list2 = [val for val in df_open['State'].unique() if val not in below_10['State'].unique()]
print(not_in_list2)

# #understanding the number of landfills lacking specific data.
# print(len(df_open['Landfill Design Capacity (tons)'].dropna()),
#       len(df_open['Waste in Place (tons)'].dropna()),
#       len(df_open['Annual Waste Acceptance Rate (tons per year)'].dropna()))
# df_open['Years until Fill'].dropna()

#exporting data for those without any year to fill data
df_na = df_open[df_open['Years until Fill'].isna()]
df_na
df_na.to_csv("No Years to Fill.csv")


['AK', 'GU', 'VT', 'WY']


In [ ]:
below_10

,County,State,Longitude,Latitude,Landfill Closure Year,Current Landfill Status,Design Landfill Area (acres),Current Landfill Area (acres),Design Landfill Depth (feet),Current Landfill Depth (feet),Landfill Design Capacity (tons),Waste in Place (tons),Annual Waste Acceptance Rate (tons per year),Designed Volume (acres^3),Current Volume Current (acres^3),Percent Volume Available,Years until Fill,Years to Planned Closure (from 2023),Difference in Years to fill and planned closure
11,Tuscaloosa,AL,-87.665850,33.243140,2037.0,Open,220.0,80.3,NaN,NaN,9826872.0,7978193.0,352153.0,NaN,NaN,NaN,5.25,14.0,-8.75
20,Coffee,AL,-86.000975,31.507053,2048.0,Open,450.0,61.6,NaN,60.0,6765340.0,4471043.0,256681.0,NaN,0.084848,NaN,8.94,25.0,-16.06
22,Cullman,AL,-86.900690,34.028780,2040.0,Open,NaN,29.0,NaN,60.0,2937045.0,2062592.0,97965.0,NaN,0.039945,NaN,8.93,17.0,-8.07
30,Baldwin,AL,-87.776100,30.457270,2036.0,Open,96.0,47.9,NaN,10.0,6408237.0,4541582.0,204333.0,NaN,0.010996,NaN,9.14,13.0,-3.86
33,Jefferson,AL,-86.815000,33.585000,2030.0,Open,NaN,149.0,60.0,NaN,9644756.0,7781050.0,7034.0,NaN,NaN,NaN,264.96,7.0,257.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,Outagamie,WI,-88.348700,44.287200,2023.0,Open,460.0,130.4,65.0,NaN,NaN,13681535.0,773008.0,0.686408,NaN,NaN,NaN,0.0,NaN
2594,Monroe,WI,-90.578000,43.873000,NaN,Open,NaN,5.0,NaN,75.0,570600.0,244351.0,40206.0,NaN,0.008609,NaN,8.11,NaN,NaN
2598,Eau Claire,WI,-91.367200,44.837500,2029.0,Open,80.0,55.0,NaN,NaN,17078157.0,10863408.0,324347.0,NaN,NaN,NaN,19.16,6.0,13.16
2611,Kanawha,WV,-81.618540,38.312790,2027.0,Open,NaN,80.8,NaN,NaN,6587686.0,5539030.0,195001.0,NaN,NaN,NaN,5.38,4.0,1.38


In [ ]:
statelist = counts['State']
totaldict = {}
for state in statelist:
  subset = df_open[df_open['State'] == f'{state}'].copy()
  totaldict[f'{state}'] = len(subset['State'])
Landfills_Open_By_State = pd.DataFrame(totaldict, index=[0]).transpose().reset_index()

# Rename the columns
Landfills_Open_By_State.columns = ['State', 'Total']
Landfills_Open_By_State


merged_df = pd.merge(counts, Landfills_Open_By_State, on='State', how='inner')

merged_df['Percentage Closing'] = round(merged_df['Number of Landfills Closing in the Next 10 Years'] / merged_df['Total'], 3)
merged_df

latex_code = merged_df.to_latex(index=False)
latex_code

with open('output_table.tex', 'w') as f:
    f.write(latex_code)


<ipython-input-22-88a364663cc9>:18: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_code = merged_df.to_latex(index=False)
